## Example: NER + SFT + DPO

### Setup

In [1]:
import json
import os
import tempfile
import time
import warnings
from pathlib import Path
from pprint import pprint
from typing import Any, Dict, List, Optional

import numpy as np
import openai
import toml
from clickhouse_connect import get_client
from IPython.display import clear_output
from minijinja import Environment

In [ ]:
CONFIG_PATH = "config/tensorzero.toml"
FUNCTION_NAME = "extract_entities_combined"
TEMPLATE_VARIANT_NAME = "gpt_4o_mini_base"
MODEL_NAME = "gpt-4o-2024-08-06"
VAL_FRACTION = 0.2

assert "OPENAI_API_KEY" in os.environ
assert "TENSORZERO_CLICKHOUSE_URL" in os.environ

openai_client = openai.OpenAI()
clickhouse_client = get_client(dsn=os.environ["TENSORZERO_CLICKHOUSE_URL"])

### STEP 1: SFT

### STEP 2: DPO

In [ ]:
config_path = Path(CONFIG_PATH)

assert config_path.exists(), f"{CONFIG_PATH} does not exist"
assert config_path.is_file(), f"{CONFIG_PATH} is not a file"

with config_path.open("r") as f:
    config = toml.load(f)


Ensure that the function and variant being fine-tuned are present in the provided config.

In [ ]:
assert "functions" in config, "No `[functions]` section found in config"
assert "variants" in config["functions"][FUNCTION_NAME], (
    f"No variants section found for function `{FUNCTION_NAME}`"
)
assert TEMPLATE_VARIANT_NAME in config["functions"][FUNCTION_NAME]["variants"], (
    f"No variant named `{TEMPLATE_VARIANT_NAME}` found in function `{FUNCTION_NAME}`"
)

Retrieve the configuration for the variant with the templates we will use for fine-tuning.

In [ ]:
function_type = config["functions"][FUNCTION_NAME]["type"]
variant = config["functions"][FUNCTION_NAME]["variants"][TEMPLATE_VARIANT_NAME]

In [ ]:
templates = {}

if "assistant_template" in variant:
    assistant_template_path = config_path.parent / variant["assistant_template"]
    with assistant_template_path.open("r") as f:
        templates["assistant"] = f.read()

if "system_template" in variant:
    system_template_path = config_path.parent / variant["system_template"]
    with system_template_path.open("r") as f:
        templates["system"] = f.read()

if "user_template" in variant:
    user_template_path = config_path.parent / variant["user_template"]
    with user_template_path.open("r") as f:
        templates["user"] = f.read()

env = Environment(templates=templates)

Initialize the ClickHouse client.

In [ ]:
assert "TENSORZERO_CLICKHOUSE_URL" in os.environ, (
    "TENSORZERO_CLICKHOUSE_URL environment variable not set"
)

clickhouse_client = get_client(dsn=os.environ["TENSORZERO_CLICKHOUSE_URL"])

Determine the ClickHouse table name for the function.

In [ ]:
inference_table_name = {"json": "JsonInference"}.get(function_type)

if inference_table_name is None:
    raise ValueError(f"Unsupported function type: {function_type}")

Query ClickHouse for inference, feedback, and metric.

In [ ]:
query = f"""
SELECT
    i.variant_name AS variant,
    i.episode_id AS episode_id,
    i.input AS input,
    i.output AS non_preferred_output,
    d.value AS preferred_output
FROM
    {inference_table_name} AS i
INNER
    JOIN DemonstrationFeedback AS d ON i.id = d.inference_id
WHERE
    (i.function_name = %(function_name)s)
LIMIT %(max_samples)s
"""

params = {
    "max_samples": MAX_SAMPLES,
    "function_name": FUNCTION_NAME,
}

df = clickhouse_client.query_df(query, params)

df.head()

## (Data format would go here)

Upload the prepared datasets to OpenAI.

In [ ]:
def upload_dataset_to_openai(df, openai_client) -> str:
    with tempfile.NamedTemporaryFile(mode="w", suffix=".jsonl", delete=False) as f:
        for item in df["openai_messages"]:
            json.dump(item, f)
            f.write("\n")
        f.flush()

        print(f"File persisted on path [{f.name}]")

        with open(f.name, "rb") as file:
            file_object = openai_client.files.create(file=file, purpose="fine-tune")

        return file_object.id


openai_client = openai.OpenAI()

dpo_fine_tuning_object_id = upload_dataset_to_openai(train_df, openai_client)
val_file_object_id = upload_dataset_to_openai(val_df, openai_client)

Launch the fine-tuning job and wait for it to complete.

In [ ]:
fine_tuning_job = openai_client.fine_tuning.jobs.create(
    training_file=dpo_fine_tuning_object_id,
    validation_file=val_file_object_id,
    model=MODEL_NAME,
    method={
        "type": "dpo",
        "dpo": {
            "hyperparameters": {"beta": 0.2},
        },
    },
)

while True:
    clear_output(wait=True)

    try:
        job_status = openai_client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
        pprint(job_status.to_dict())
        if job_status.status in ("succeeded", "failed", "cancelled"):
            break
    except Exception as e:
        print(f"Error: {e}")

    time.sleep(10)

print(f"The fine-tuning job has compeleted with result {job_status.status}")

TODO: Adding the fine-tuned model to the config file
TODO: Adding a new variant to your function to use the fine-tuned model
